# Prototype - *get_patch_location_array()* function
    * Common to all 4 patch extraction use cases.
    * Decouples selection image mask & (mask size) from individual use case flows.
    * Insertion point for new or modified location-selection methods.
    * Allow inspection / select-method parameter modification before making output.
    * Allows paralellization of patch location (think slower more specific select methods). <br>
****
## Assemble and create unit test data:
### Redo unit test for **get_sample_selection_mask()** (output type change)
****
## Review Existing Selection Algorithm usages
```python
#                   common to both
grey_thumbnail = np.array(thumbnail.convert("L"))
thresh = threshold_otsu(grey_thumbnail)
mask = np.array(grey_thumbnail) < thresh

#                   new in opneslide_2_tfrecord.py
thresh = 80
np_img = np.array(one_thumb.convert('RGB'))
np_img = rgb2lab(np_img)
np_img = np_img[:,:,0]
mask_im = np.array(np_img) < thresh
```
```python
"""
                    87: wsi-sampler/sampler.py
"""
if np.sum(mask[x:x_mask_window, y:y_mask_window]) > 0:
    # include this location ...
```
```python
"""
                    504: DigiPath_MLTK/src/python/openslide_2_tfrecord.py
"""
mask_value = np.float(np.sum(thumb_arr==0)) / np.float(np.prod(thumb_arr.shape))
if mask_value <= drop_threshold:
    # include this location ... 
```
****
****
### Proto function to sleep on
#### Args:
    * wsi_image_name
    * patch_height
    * patch_width
    * patch_selection_method
    * patch_selection_parameter_1, 2, 3,...
        * variables that begin with "patch_selection_parameter_"
        * allow new methods with specific patch select parameters 
    * max_cpus - limit to allow other sections to run <br>
#### Returns:
    * patch selected array
    * rejected patches array <br>
****
### Sleep on it  <:- )
```python
def get_patch_location_array(run_parameters):
    """ temporary directories hold scattered thread outputs """    
    select_list = []
    reject_list = []
    # common for parallel: temp_dir_reject_locations, temp_dir_select_locations
    temp_dir_reject_locations = TemporaryDirectory()
    temp_dir_select_locations = TemporaryDirectory()
    def threshold_otsu_selector():
        # get the mask
        # get the rows & cols array
        # parallelize on selection-worker
        
    def threshold_rgb2lab_selector():
        # get the mask
        # get the rows & cols array
        # parallelize on selection-worker
        
    SELECT_LOCATION_METHOD = {'threshold_otsu', threshold_otsu_selector, 
                              'threshold_rgb2lab': threshold_rgb2lab_selector}
    
    patch_select_method = run_parameters['patch_select_method']
    
    if patch_select_method in SELECT_LOCATION_METHOD:
        rc = SELECT_LOCATION_METHOD[patch_select_method](run_parameters)
    
    # collect function: merge all temp files into a list
    if rc == 0:
        # collect the mini-files into the select and reject location lists
        select_list = list_from_temp_dir(temp_dir_select_locations)
        reject_list = list_from_temp_dir(temp_dir_reject_locations)
        
    # cleanup
    temp_dir_reject_locations.cleanup()
    temp_dir_select_locations.cleanup()
    
    return select_list, reject_list
```
### First Edit:
```python
def get_patch_location_array(run_parameters):
    """ Usage: patch_location_array = get_patch_location_array(run_parameters)
    Args:
        run_parameters:     keys:
                            image_file_name,
                            thumbnail_divisor,
                            patch_select_method,
                            patch_height,
                            patch_width
    Returns:
        patch_location_array
        
    """
    patch_location_array = []
    
    image_file_name = run_parameters['image_file_name']
    thumbnail_divisor = run_parameters['thumbnail_divisor']
    patch_select_method = run_parameters['patch_select_method']
    patch_height = run_parameters['patch_height']
    patch_width = run_parameters['patch_width']
    
    #                     OpenSlide open                      #
    os_im_obj = openslide.OpenSlide(image_file_name)
    
    pixels_height = os_im_obj.dimensions[1]
    rows_fence_array = get_fence_array(patch_length=patch_height, overall_length=pixels_height)
    
    pixels_width = os_im_obj.dimensions[0]
    cols_fence_array = get_fence_array(patch_length=patch_width, overall_length=pixels_width)
        
    small_im = os_im_obj.get_thumbnail((pixels_height // thumbnail_divisor, 
                                        pixels_width // thumbnail_divisor))
    os_im_obj.close()
    #                     OpenSlide close                     #
    
    mask_im = get_sample_selection_mask(small_im, patch_select_method)
        
    it_rows = zip(rows_fence_array[:,0] // thumbnail_divisor, 
                  rows_fence_array[:,1] // thumbnail_divisor, 
                  rows_fence_array[:,0])
    
    lft_cols = cols_fence_array[:,0] // thumbnail_divisor
    rgt_cols = cols_fence_array[:,1] // thumbnail_divisor
    cols_array = cols_fence_array[:,0]
    
    for tmb_row_top, tmb_row_bot, row_n in it_rows:
        it_cols = zip(lft_cols, rgt_cols, cols_array)
        for tmb_col_lft, tmb_col_rgt, col_n in it_cols:
            if (mask_im[tmb_row_top:tmb_row_bot, tmb_col_lft:tmb_col_rgt]).sum() > 0:
                patch_location_array.append((row_n, col_n))
                
    return patch_location_array
```

In [5]:
import time
import os
import sys
import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

data_dir = '../../DigiPath_MLTK_data/Aperio'
file_type_list=['.svs', '.tif', '.tiff']
fs_od = get_file_size_ordered_dict(data_dir, file_type_list)
list_number = 0
for k, v in fs_od.items():
    print('%3i %30s: %i'%(list_number, k,v))
    list_number += 1

  0         CMU-1-Small-Region.svs: 1938955
  1               JP2K-33003-1.svs: 63847265
  2           CMU-1-JP2K-33005.svs: 132565343
  3                      CMU-1.svs: 177552579
  4                      CMU-3.svs: 253815723
  5               JP2K-33003-2.svs: 289250433
  6                      CMU-2.svs: 390750635


In [6]:
run_parameters = {}

data_dir = '../../DigiPath_MLTK_data/Aperio/'
im_files_list = list(fs_od.keys())

run_parameters['image_file_name'] = os.path.join(data_dir, im_files_list[6])
print('Image File:\n', run_parameters['image_file_name'])

run_parameters['thumbnail_divisor'] = 40
run_parameters['patch_select_method'] = 'threshold_rgb2lab' # 'threshold_otsu'
run_parameters['patch_height'] = 224
run_parameters['patch_width'] = 224

"""
                output & time test the function
"""
t0 = time.time()
patch_location_array = get_patch_location_array(run_parameters)
tt = time.time() - t0

print('%i images found\t%0.3f s\n'%(len(patch_location_array), tt))
for d in patch_location_array:
    print(d)

Image File:
 ../../DigiPath_MLTK_data/Aperio/CMU-2.svs
2489 images found	0.449 s

(783, 7192)
(783, 7416)
(783, 7640)
(783, 7864)
(783, 8088)
(783, 8312)
(783, 8536)
(783, 8760)
(1007, 6520)
(1007, 6744)
(1007, 6968)
(1007, 7192)
(1007, 7416)
(1007, 7640)
(1007, 7864)
(1007, 8088)
(1007, 8312)
(1007, 8536)
(1007, 8760)
(1007, 8984)
(1007, 9208)
(1231, 4504)
(1231, 4728)
(1231, 4952)
(1231, 5176)
(1231, 5400)
(1231, 5624)
(1231, 5848)
(1231, 6072)
(1231, 6296)
(1231, 6520)
(1231, 6744)
(1231, 6968)
(1231, 7192)
(1231, 7416)
(1231, 7640)
(1231, 7864)
(1231, 8088)
(1231, 8312)
(1231, 8536)
(1231, 8760)
(1231, 8984)
(1231, 9208)
(1455, 3832)
(1455, 4056)
(1455, 4280)
(1455, 4504)
(1455, 4728)
(1455, 4952)
(1455, 5176)
(1455, 5400)
(1455, 5624)
(1455, 5848)
(1455, 6072)
(1455, 6296)
(1455, 6520)
(1455, 6744)
(1455, 6968)
(1455, 7192)
(1455, 7416)
(1455, 7640)
(1455, 7864)
(1455, 8088)
(1455, 8312)
(1455, 8536)
(1455, 8760)
(1455, 8984)
(1455, 9208)
(1455, 27352)
(1679, 3160)
(1679, 3384)
(1

(7727, 5400)
(7727, 5624)
(7727, 5848)
(7727, 6072)
(7727, 6296)
(7727, 6520)
(7727, 6744)
(7727, 6968)
(7727, 7192)
(7727, 7416)
(7727, 7640)
(7727, 7864)
(7727, 8088)
(7727, 8312)
(7727, 8536)
(7727, 8760)
(7727, 8984)
(7727, 9208)
(7727, 9432)
(7727, 9656)
(7727, 9880)
(7727, 10104)
(7727, 10328)
(7727, 10552)
(7727, 10776)
(7727, 11000)
(7727, 14584)
(7727, 19288)
(7727, 19512)
(7727, 19736)
(7727, 19960)
(7727, 20184)
(7727, 20408)
(7727, 20632)
(7727, 20856)
(7727, 21080)
(7727, 21304)
(7727, 21528)
(7727, 21752)
(7727, 21976)
(7727, 22200)
(7727, 22424)
(7727, 22648)
(7727, 22872)
(7727, 23096)
(7727, 23320)
(7727, 23544)
(7727, 23768)
(7727, 23992)
(7727, 24216)
(7727, 24440)
(7727, 24664)
(7727, 24888)
(7727, 25112)
(7727, 25336)
(7727, 25560)
(7727, 25784)
(7727, 26008)
(7727, 26232)
(7727, 26456)
(7727, 26680)
(7727, 26904)
(7727, 27128)
(7727, 27352)
(7727, 27576)
(7727, 28024)
(7951, 3160)
(7951, 3384)
(7951, 3608)
(7951, 3832)
(7951, 4056)
(7951, 4280)
(7951, 4504)
(7951,